As noted in Tutorial 00, `pygsti` can provide many reports or presentation slides.

In this tutorial, we look at how to customize some of the output of `pygsti`'s report generating module.

In [1]:
#Import the GST module -- you probably want this at the beginning of every notebook
import pygsti
import json

Creating reports is a method of the `pygsti.report.Results` object. Below, we create such an object, and then initialize it with the appropriate fiducials, germs, maximum lengths, etc.

It's important to note that the `pygsti.report.Results` object requires an argument to indicate which objective function you wish to score the estimate using. It's recommended that, if you ran MC2GST, you use the `"chi2"` argument, while if you ran MLGST, you use `"logL"`. Otherwise, you will score your estimate based on an objective function you weren't optimizing over, so your scores may be rather weird!

Creating reports is a method of the `pygsti.report.Results` object. Below, we create such an object, and then initialize it with the appropriate fiducials, germs, maximum lengths, etc.

It's important to note that the `pygsti.report.Results` object requires an argument to indicate which objective function you wish to score the estimate using. It's recommended that, if you ran MC2GST, you use the `"chi2"` argument, while if you ran MLGST, you use `"logL"`. Otherwise, you will score your estimate based on an objective function you weren't optimizing over, so your scores may be rather weird!

First, we'll make smaller reports that don't contain any confidence intervals for the estimates,
nor any appendices.

NOTE: In order for you to create PowerPoint files, you'll have to have `python-pptx` installed on your system.

In [2]:
# Follow Algorithm tutorial to generate LSGST gatesets
gs_target = pygsti.io.load_gateset("tutorial_files/Example_Gateset.txt")
ds = pygsti.io.load_dataset("tutorial_files/Example_Dataset.txt", cache=True)
fiducialList = pygsti.io.load_gatestring_list("tutorial_files/Example_FiducialList.txt")

#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
specs = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
gs_lgst = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=1)

#Gauge optimize the result to match the target gateset
gs_lgst_after_gauge_opt = pygsti.optimize_gauge(gs_lgst, "target", targetGateset=gs_target)

#Contract the result to CPTP
gs_clgst = pygsti.contract(gs_lgst_after_gauge_opt, "CPTP")

#Get lists of gate strings for successive iterations of MC2GST to use
specs  = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
germList = pygsti.io.load_gatestring_list("tutorial_files/Example_GermsList.txt")
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))
lsgstListOfLists = [ pygsti.io.load_gatestring_list("tutorial_files/Example_LSGSTlist%d.txt" % l) for l in maxLengthList]
        
gs_lsgst_list = pygsti.do_iterative_mc2gst(ds, gs_clgst, lsgstListOfLists, verbosity=2,
                                         minProbClipForWeighting=1e-6, probClipInterval=(-1e6,1e6),
                                         returnAll=True )

go_gatesets = [ pygsti.optimize_gauge(gs, "target", targetGateset=gs_target, gateWeight=1, spamWeight=0.001) for gs in gs_lsgst_list]

Loading from cache file:  tutorial_files/Example_Dataset.txt.cache



--- Iterative LSGST: Beginning iter 1 of 10 : 92 gate strings ---


--- Least Squares GST ---
  Sum of Chi^2 = 48.2156 (92 data params - 40 model params = expected mean of 52; p-value = 0.623488)



--- Iterative LSGST: Beginning iter 2 of 10 : 92 gate strings ---


--- Least Squares GST ---
  Sum of Chi^2 = 48.2156 (92 data params - 40 model params = expected mean of 52; p-value = 0.623489)

...

--- Least Squares GST ---
  Sum of Chi^2 = 2041.44 (1969 data params - 40 model params = expected mean of 1929; p-value = 0.0369745)



--- Iterative LSGST: Beginning iter 9 of 10 : 2353 gate strings ---


--- Least Squares GST ---
  Sum of Chi^2 = 2430 (2353 data params - 40 model params = expected mean of 2313; p-value = 0.0444015)



--- Iterative LSGST: Beginning iter 10 of 10 : 2737 gate strings ---


--- Least Squares GST ---
  Sum of Chi^2 = 2799.71 (2737 data params - 40 model params = expected mean of 2697; p-value = 0.082232)

In [3]:
res = pygsti.report.Results()
res.init_Ls_and_germs("chi2", gs_target, ds, gs_clgst, maxLengthList, germList,
                    go_gatesets, lsgstListOfLists, fiducialList, fiducialList, 
                    pygsti.construction.repeat_with_max_length, False)

In [4]:
#Make smaller reports (no confidence intervals or appendicies)
res.create_full_report_pdf(filename="tutorial_files/Example_reportB.pdf", verbosity=2, confidenceLevel=None,
                        debugAidsAppendix=False, gaugeOptAppendix=False,
                        pixelPlotAppendix=False, whackamoleAppendix=False)

res.create_brief_report_pdf(filename="tutorial_files/Example_briefB.pdf", verbosity=2, confidenceLevel=None)

res.create_presentation_pdf(filename="tutorial_files/Example_slidesB.pdf", verbosity=2, confidenceLevel=None,
                         debugAidsAppendix=False, pixelPlotAppendix=False, whackamoleAppendix=False)

res.create_presentation_ppt(filename="tutorial_files/Example_slidesB.ppt", verbosity=2, confidenceLevel=None,
                         debugAidsAppendix=False, pixelPlotAppendix=False, whackamoleAppendix=False)

*** Generating tables ***


Generating table: targetSpamTable


Generating table: targetGatesTable


Generating table: datasetOverviewTable


Generating table: bestGatesetSpamTable


...

Latexing fiducialListTable table...


Latexing germListTable table...


Latexing datasetOverviewTable table...


Final output PPT tutorial_files/Example_slidesB.pptx successfully generated.


/home/enielse/research/pyGSTi/packages/pygsti/report/plotting.py:357: RuntimeWarning: invalid value encountered in greater
  return_value = _np.where(_np.greater(norm_trans, lin_norm_value),
/usr/lib64/python2.7/site-packages/numpy/ma/core.py:790: RuntimeWarning: invalid value encountered in greater_equal
  return umath.absolute(a) * self.tolerance >= umath.absolute(b)
/usr/lib64/python2.7/site-packages/numpy/ma/core.py:802: RuntimeWarning: invalid value encountered in less_equal
  return umath.less_equal(x, self.critical_value)


In [5]:
res.create_full_report_pdf(filename="tutorial_files/Example_report.pdf", verbosity=2, confidenceLevel=95,
                        debugAidsAppendix=True, gaugeOptAppendix=True,
                        pixelPlotAppendix=True, whackamoleAppendix=True)

res.create_brief_report_pdf(filename="tutorial_files/Example_brief.pdf", verbosity=2, confidenceLevel=95)

res.create_presentation_pdf(filename="tutorial_files/Example_slides.pdf", verbosity=2, confidenceLevel=95,
                         debugAidsAppendix=True, pixelPlotAppendix=True, whackamoleAppendix=True)

res.create_presentation_ppt(filename="tutorial_files/Example_slides.ppt", verbosity=2, confidenceLevel=95,
                         debugAidsAppendix=True, pixelPlotAppendix=True, whackamoleAppendix=True)

*** Generating tables ***


Generating table: targetSpamTable (w/95% CIs)


Generating table: targetGatesTable (w/95% CIs)


Generating table: datasetOverviewTable (w/95% CIs)


Generating table: bestGatesetSpamTable (w/95% CIs)


...

Latexing fiducialListTable table...


Latexing germListTable table...


Latexing datasetOverviewTable table...


Final output PPT tutorial_files/Example_slides.pptx successfully generated.


/home/enielse/research/pyGSTi/packages/pygsti/report/plotting.py:407: RuntimeWarning: invalid value encountered in greater
  resdat[resdat>0] /= abs(vmax - midpoint)
/home/enielse/research/pyGSTi/packages/pygsti/report/plotting.py:408: RuntimeWarning: invalid value encountered in less
  resdat[resdat<0] /= abs(vmin - midpoint)


In [6]:
import pygsti.report.latex as LU
reload(LU)
LU.latex_value(1.2e30,2)

'1\\e{30}'